In [7]:
"""
This file contains hyperparameter optimization for SIFT and SVC model. They are used in the first part of the project.
The whole model training and testing process is done in SIFT_feature_matching.py file.

Yontem - 1
Feature extraction using SIFT and Bag of Visual Words (BOVW) model
Feature extraction is done by SIFT.
Bag of Visual Words (BOVW) model is used to represent images as histograms of visual words.
Extracted features are then used to train a Support Vector Machine (SVM) model for image classification.
"""


# Imports
import cv2
import numpy as np
import os
import random
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, precision_recall_curve
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [8]:
# Functions
def extract_features(image_paths, extractor):
    features = []
    for path in image_paths:
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = extractor.detectAndCompute(gray, None)
        if descriptors is not None:
            features.append(descriptors)
    return np.concatenate(features, axis=0)

def create_bovw_histograms(image_paths, extractor, kmeans):
    histograms = []
    for path in image_paths:
        image = cv2.imread(path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        keypoints, descriptors = extractor.detectAndCompute(gray, None)
        histogram = np.zeros(len(kmeans.cluster_centers_))  # Initialize histogram for each image
        if descriptors is not None:
            labels = kmeans.predict(descriptors)
            for label in labels:
                histogram[label] += 1
        histograms.append(histogram)  # Append histogram regardless of descriptor availability
    return np.array(histograms)

def load_dataset(root_dir): # takes the folder name (class name) as label
    image_paths = []
    labels = []
    label_to_id = {}
    for idx, class_name in enumerate(os.listdir(root_dir)):
        class_path = os.path.join(root_dir, class_name)
        if os.path.isdir(class_path):
            label_to_id[class_name] = idx
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                image_paths.append(file_path)
                labels.append(idx)
    return image_paths, labels, label_to_id

def model_evaluation(X, y, y_pred):
    # Match Accuracy
    match_accuracy = np.mean(y_pred == y)
    print('Match Accuracy:', match_accuracy)

    # Matching Precision and Recall
    precision = precision_score(y, y_pred, average='macro')
    print('Precision:', precision)
    recall = recall_score(y, y_pred, average='macro')
    print('Recall:', recall)

    # Feature Count
    feature_count = X.shape[1]
    print('Feature Count:', feature_count)

    # Unique Match Ratio
    unique_match_ratio = len(np.unique(y_pred)) / len(np.unique(y))
    print('Unique Match Ratio:', unique_match_ratio)

    return {
        'Match Accuracy': match_accuracy,
        'Precision': precision,
        'Recall': recall,
        'Feature Count': feature_count,
        'Unique Match Ratio': unique_match_ratio
    }

def display_all_results(image_paths, predicted_labels, true_labels, label_to_id):
    id_to_label = {v: k for k, v in label_to_id.items()}
    num_images = len(image_paths)
    num_cols = 4
    num_rows = num_images // num_cols + (1 if num_images % num_cols != 0 else 0)

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows))
    for idx, path in enumerate(image_paths):
        row = idx // num_cols
        col = idx % num_cols
        ax = axes[row, col] if num_rows > 1 else axes[col]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        pred_label = id_to_label.get(predicted_labels[idx], "Unknown")
        true_label = id_to_label.get(true_labels[idx], "Unknown")
        result = "TRUE" if predicted_labels[idx] == true_labels[idx] else "FALSE"
        ax.set_title(f"Predicted: {pred_label}, True: {true_label}\n{result}")
        ax.axis('off')
    plt.tight_layout()
    plt.show()

def display_sample_results(image_paths, predicted_labels, true_labels, label_to_id, sample_size=10):
    id_to_label = {v: k for k, v in label_to_id.items()}
    sample_size = min(sample_size, len(image_paths))
    
    # Randomly sample indices without replacement
    sampled_indices = random.sample(range(len(image_paths)), sample_size)
    
    # Subset the data
    sampled_image_paths = [image_paths[i] for i in sampled_indices]
    sampled_predicted_labels = [predicted_labels[i] for i in sampled_indices]
    sampled_true_labels = [true_labels[i] for i in sampled_indices]
    
    num_cols = 4
    num_rows = sample_size // num_cols + (1 if sample_size % num_cols != 0 else 0)
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 5 * num_rows), squeeze=False)
    axes = axes.flatten()
    
    for idx, path in enumerate(sampled_image_paths):
        ax = axes[idx]
        image = cv2.imread(path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)
        pred_label = id_to_label.get(sampled_predicted_labels[idx], "Unknown")
        true_label = id_to_label.get(sampled_true_labels[idx], "Unknown")
        result = "TRUE" if sampled_predicted_labels[idx] == sampled_true_labels[idx] else "FALSE"
        ax.set_title(f"Predicted: {pred_label}, True: {true_label}\n{result}")
        ax.axis('off')
    
    # Hide any unused subplot areas
    for idx in range(len(sampled_image_paths), len(axes)):
        axes[idx].axis('off')
    
    plt.tight_layout()
    plt.show()

In [9]:
# Parameters
num_clusters = 30
root_dir_train = 'data_128x128/train'
root_dir_test = 'data_128x128/test'
root_dir_val = 'data_128x128/validation'

# Load datasets
image_paths_train, labels_train, label_to_id_train = load_dataset(root_dir_train)
image_paths_test, labels_test, label_to_id_test = load_dataset(root_dir_test)
image_paths_val, labels_val, label_to_id_val = load_dataset(root_dir_val)

In [10]:
""""
Parameter Optimization for SIFT
Checks for the best settings for SIFT in training images

"""
train_folder = root_dir_train

# Ranges for parameters to optimize
nfeatures_range = [0, 50, 100, 200]
nOctaveLayers_range = [3, 4, 5]
contrastThreshold_range = [0.04, 0.1, 0.2]
edgeThreshold_range = [10, 15, 20]
sigma_range = [1.2, 1.6, 2.0]

best_score = 0
best_settings = {}

# Loop through each class in the train folder
for class_folder in os.listdir(train_folder):
    class_path = os.path.join(train_folder, class_folder)
    
    # List the images in the class folder
    images = os.listdir(class_path)

    for image in images:
        if image.endswith('.jpg'):
            image_path = os.path.join(class_path, image)
            image = cv2.imread(image_path, 0)

            # Search for each image
            for nfeatures in nfeatures_range:
                for nOctaveLayers in nOctaveLayers_range:
                    for contrastThreshold in contrastThreshold_range:
                        for edgeThreshold in edgeThreshold_range:
                            for sigma in sigma_range:
                                # Create SIFT with current settings
                                sift = cv2.SIFT_create(nfeatures=nfeatures, nOctaveLayers=nOctaveLayers,
                                                    contrastThreshold=contrastThreshold, edgeThreshold=edgeThreshold,
                                                    sigma=sigma)

                                # Detect features
                                keypoints = sift.detect(image, None)

                                # Use the number of detected keypoints as the score
                                score = len(keypoints)

                                # Update best score and settings if current score is better
                                if score > best_score:
                                    best_score = score
                                    best_settings = {
                                        'nfeatures': nfeatures,
                                        'nOctaveLayers': nOctaveLayers,
                                        'contrastThreshold': contrastThreshold,
                                        'edgeThreshold': edgeThreshold,
                                        'sigma': sigma,
                                        'image': image
                                    }

# Best settings for SIFT 
print("Best score:", best_score)
print("Best settings:", best_settings)

Best score: 344
Best settings: {'nfeatures': 0, 'nOctaveLayers': 5, 'contrastThreshold': 0.04, 'edgeThreshold': 10, 'sigma': 1.2, 'image': array([[126, 130,  77, ...,  36,  22,  49],
       [ 99, 135, 111, ...,  26,  20,  20],
       [ 31, 137, 141, ...,  75,  48,  46],
       ...,
       [127, 135,  76, ...,  86, 163, 115],
       [148, 161, 105, ..., 128, 159,  74],
       [171, 145, 135, ..., 114,  88,  59]], dtype=uint8)}


In [5]:
# SIFT feature extractor and KMeans clustering model creation
sift = cv2.SIFT_create()
features_train = extract_features(image_paths_train, sift)
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(features_train)

c:\Users\Emre\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


KMeans(n_clusters=100)

In [6]:
# Create BOVW histograms for train, test, and validation sets
bovw_histograms_train = create_bovw_histograms(image_paths_train, sift, kmeans)
bovw_histograms_test = create_bovw_histograms(image_paths_test, sift, kmeans)
bovw_histograms_val = create_bovw_histograms(image_paths_val, sift, kmeans)

# Train, test, and validation data split
X_train, y_train = bovw_histograms_train, labels_train
X_test, y_test = bovw_histograms_test, labels_test
X_val, y_val = bovw_histograms_val, labels_val

In [8]:
""""
Parameter Optimization for SVC
Checks for the best parameters for SVC in training images

"""

param_grid = {
    'svc__C': [0.1, 1, 10, 100],
    'svc__kernel': ['linear', 'rbf'],
    'svc__gamma': ['scale', 'auto']
}

pipeline = make_pipeline(StandardScaler(), SVC())
grid_search = GridSearchCV(pipeline, param_grid, cv=5, n_jobs=-1, verbose=1)

grid_search.fit(X_train, y_train)

# Best parameters found during grid search for SVC
print("Best Parameters:", grid_search.best_params_)

# Best cross-validation accuracy
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred_test = best_model.predict(X_test)

# Evaluate the best model on the test set
print("Model Evaluation on Test Set after Hyperparameter Tuning")
test_evaluation_after_tuning = model_evaluation(X_test, y_test, y_pred_test)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'svc__C': 10, 'svc__gamma': 'scale', 'svc__kernel': 'rbf'}
Best Cross-Validation Accuracy: 0.19276190476190477
Model Evaluation on Test Set after Hyperparameter Tuning
Match Accuracy: 0.069
Precision: 0.067575152463602
Recall: 0.046000000000000006
Feature Count: 100
Unique Match Ratio: 1.5


c:\Users\Emre\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
